To better understand feasibility of reusing the netiob scripts for extracting T1DEXI, we analyze the output of the netiob scripts that process T1DExi data and summarize out findings. 

In [65]:
import pandas as pd
import pyreadstat
import re
from datetime import datetime, timedelta
data_datetime_list = ['RFICDTC', 'BRTHDTC', 'DXDTC', 'LBDTC', 'MLDTC', 'NVDTC', 'NVENDTC', 'PRSTDTC', 'QSDTC', 
                                   'PRDTC', 'SCDTC', 'RPDTC', 'DMDTC', 'CMSTTPT', 'CMENTPT', 'VSDTC', 'VSENDTC', 'FADTC', 'DATETIME']
        

def convert_timestamp(timestamps_col):
    """
    Converts timestamp columns in the input DataFrame from seconds since 1960-01-01
    to the format 'YYYY-MM-DD HH:MM:SS'

    Args:
    timestamps_col : timestamp column(s) to be converted
    
    Returns:
    converted_timestamp: (Datetime) timestamp column(s) converted to 'YYYY-MM-DD HH:MM:SS' format.
    """
    # Check the type of timestamps_col (If it's a Series and not of type pd.Timestamp)
    if isinstance(timestamps_col, pd.Series) and timestamps_col.dtype != pd.Timestamp:
        # Error handling
        try:
            # Convert timestamp to datetime format if not null or NaT
            timestamps_col = timestamps_col.apply(lambda x: datetime(1960, 1, 1) + timedelta(seconds=x) if pd.notnull(x) else pd.NaT)
        except TypeError:
            timestamps_col = pd.to_datetime(timestamps_col)
    
    # Otherwise if timestamps_col is in float format
    elif isinstance(timestamps_col, float):
        timestamps_col = datetime(1960, 1, 1) + timedelta(seconds=timestamps_col)

    return timestamps_col

def clean_dataframe(data):
    """
    Cleans the input DataFrame by decoding, stripping unwanted characters, and converting the timestamps.

    Args:
    - data: (DataFrame) Input DataFrame to be cleaned.

    Returns:
    - cleaned_data: (DataFrame) Cleaned DataFrame with unwanted characters removed from object-type columns,
                    rows containing unwanted characters removed, and timestamps converted.
    """
    unwanted_xters_list = ['%']     # Unwanted character list

    # Strip the object-type characters "b'" from the data
    cleaned_data = data.map(lambda x: x.decode('latin-1') if isinstance(x, bytes) else x)
    # Remove the unwanted characters
    cleaned_data = cleaned_data[~cleaned_data.isin(unwanted_xters_list).any(axis=1, skipna=True)]

    for cols in cleaned_data.columns:     # Iterate through the dataframe columns
        # Check if processing column is in the list of timestamp columns
        if cols in data_datetime_list:
            # If yes, call the implement the convert_timestamp method on it.
            cleaned_data[cols] = cleaned_data[cols].map(lambda x: convert_timestamp(x))
    
    # Return clean dataframe
    return cleaned_data

def iso8601_duration_to_timedelta(duration_str):
    """
    Converts duration in ISO8601 (P0Y0M0DT4H0M0S) format to HH:MM:SS

    Args:
    - duration_str: (str) Duration String Value

    Returns:
    - Time format (HH:MM:SS)
    """
    # Regular expression to extract components
    regex = r'P(?:([0-9]+)Y)?(?:([0-9]+)M)?(?:([0-9]+)D)?(?:T(?:([0-9]+)H)?(?:([0-9]+)M)?(?:([0-9]+)S)?)?'      
    matches = re.match(regex, duration_str)         # Match strings in the input duration string
    
    if matches:     # If there's a match            
        # Extract time related components 
        years = int(matches.group(1)) if matches.group(1) else 0
        months = int(matches.group(2)) if matches.group(2) else 0
        days = int(matches.group(3)) if matches.group(3) else 0
        hours = int(matches.group(4)) if matches.group(4) else 0
        minutes = int(matches.group(5)) if matches.group(5) else 0
        seconds = int(matches.group(6)) if matches.group(6) else 0

        return timedelta(hours=hours, minutes=minutes, seconds=seconds)     # Convert to timedelta
    else:       # If no match, return 00:00:00
        return timedelta(hours=0, minutes=0, seconds=0)

def _get_data(self, data_label):
    """
    Reads the data from a SAS file, calls the clean_dataframe() function to strip, clean, 
    dropna, and convert timestamps, and returns the cleaned DataFrame along with the count 
    of number of data subjects in the DatFrame.
    
    Note: We provided options to use XPORT or PYREADSTAT libraries. Our implementation is 
    etup to use pyreadstat

    Args:
    - DATA_PATH: (FILE_DIR) File Directory Path to T1DEXI Dataset
    - data_label: (str) Filename without .xpt. Example: 'FACM', 'ML', etc.
        
    Returns:
    - data_label_df: (DataFrame) DataFrame after cleaning, converting timestamp columns, and removing NaN rows.
    - data_subject_count: (int) Count of unique data subjects in the prepared DataFrame.
    """

    #data_label_df = pd.read_sas(f'{self.DATA_PATH}{data_label}.xpt')     # Read the entered data label file (if using XPORT).
    data_label_df, metadata = pyreadstat.read_xport(f'{self.DATA_PATH}{data_label}.xpt') # Uncomment if using pyreadstat to read data
    data_label_df = clean_dataframe(data_label_df)                  # Clean dataframe using the clean_dataframe() method
    if 'USUBJID' in data_label_df.columns:
        data_subject_count = data_label_df['USUBJID'].nunique()              # Count the number of user subjects in the dataframe
    else:
        data_subject_count = None

    # Return clean DataFrame, and user subject count
    return data_label_df, data_subject_count


def get_device_data(self, device_category=None):
    """
    Retrieves device data from the prepared DataFrame, categorizes the device type,
    and filters out certain device types, and returns (1) DataFrames with user id, 
    device type and device category, (2) DataFrame filtered by pump users category,
    and (3) list of all device types.

    Args:
    - device_category: (str) Device type, e.g., 'aid', 'non_aid'
                        Note: Default = None (i.e., all categories will be returned)
    Returns:
    - device_category_data: (DataFrame) DataFrame filtered by only the entered device category
    - all_device_list: (list) List of all device types
    """
    # Get the device type data and streamline to only two columns - 'USUBJID' and 'DXTRT'
    device_data = self._get_data('DX')[0][['USUBJID', 'DXTRT']]
    # Filter out device type 'INSLIN PUMP' and 'CLOSED LOOP INSULIN PUMP' as they are both generic device type names
    device_data = device_data[~device_data['DXTRT'].isin(['INSULIN PUMP', 'CLOSED LOOP INSULIN PUMP'])]

    # [Discretional]. We observed that subject ID: 766 device type should be 'MEDTRONIC 770G IN AUTO MODE' and we updated as such
    device_data.loc[device_data['USUBJID'] == '766', 'DXTRT'] = 'MEDTRONIC 770G IN AUTO MODE'
    # Create a new column and populate with the appropriate device category using the categorize_device_type() function
    device_data['DEVICE'] = device_data['DXTRT'].apply(self.categorize_device_type)

    # Create a fileter for the 'AID' and 'non-AID' device categories
    if device_category == 'aid':
        # Get only AID device data
        device_category_data = device_data[device_data['DXTRT'].isin(self.aid_device_list)]
    elif device_category == 'non_aid':
        # Get only non-AID device data
        # Note that all non-AID categories are combined here
        extend_non_aid_list = itertools.chain(self.non_aid_device_list, self.mdi_device_list, self.non_aid_predictive_list)
        device_category_data = device_data[device_data['DXTRT'].isin(extend_non_aid_list)]
    else:   # Get all device categories
        device_category_data = device_data
    
    # Get the list of all devices in the DataFrame
    all_device_list = device_data['DXTRT'].unique()

    # Return dataframe classified by devices categories, and list of all devices, respectively
    return device_category_data, all_device_list

# def get_insulin_data_chunk(device_category=None):
#     """
#     Retrieves Insulin data from the new cleaned, chunked, insulin 
#     DataFrame and returns (1) a streamlined insulin dataframe, 
#     and (2) insulin data based on pump users categories.

#     Args:
#     - device_category: (str) Device type, e.g., 'aid', 'non_aid'
#                         Note: Default = None (i.e., all categories will be returned)
#     Returns:
#     - device_category_insulin_df: (DatFrame) Filtered insulin data based on input device category
#     - basal_data: (DataFrame) Filtered dataframe with only basal data
#     - bolus_data: (DataFrame) Filtered dataframe with only bolus data
#     """
#     # Get the insulin data and streamline to few important columns  
#     insulin_data = _get_data(f'FACM_CHUNK_{device_category}')[0][['USUBJID', 'FATEST', 'FACAT', 'FADTC', 'FASTRESN', 'FADUR', 'INSSTYPE', 'INSNMBOL', 'INSEXBOL']]
#     # Convert the iso8601 duration format to a readable form and remove the '0 days' string prepended to the output
#     insulin_data['FADUR'] = (insulin_data['FADUR'].apply(iso8601_duration_to_timedelta)).astype(str).str.replace('0 days ', '')

#     # Round the unit values in the columns ['FASTRESN', 'INSNMBOL', 'INSEXBOL'] to 2 decimal places (takes care of scientific notation formats)  
#     insulin_data[['FASTRESN', 'INSNMBOL', 'INSEXBOL']] = (insulin_data[['FASTRESN', 'INSNMBOL', 'INSEXBOL']]).round(2)
#     insulin_data['FADTC'] = pd.to_datetime(insulin_data['FADTC'])

#     basal_data = insulin_data[insulin_data['FACAT'] == 'BASAL']     # Extract only BASAL records
#     bolus_data = insulin_data[insulin_data['FACAT'] == 'BOLUS']     # Extract only BOLUS records
    
#     # Get user ids of categories of pump users, e.g., 'aid', 'non_aid' (Depending on the input cat.)
#     #user_device_data = get_device_data(device_category=device_category)[0]['USUBJID']
#     # Filter the insulin data based on users ids  extracted for pump user categories.    
#     #device_category_insulin_df = insulin_data[insulin_data['USUBJID'].isin(user_device_data)]

#     # Optionally, if BASAL/BOLUS records based on user category is sought, uncomment the code line below,
#     # change the 'BASAL' or 'BOLUS' values accordingly, and return the variable
#     #device_category_insulin_type_df = device_category_insulin_df[device_category_insulin_df['FACAT'] == 'BASAL']

#     # Return Exercise DataFrame filtered by entered device category, basal only records, bolus only records.
#     return basal_data, bolus_data


In [67]:
chunk_path = '/T1DEXI/FACM_CHUNK.xpt'
data_label_df, metadata = pyreadstat.read_xport(chunk_path) # Uncomment if using pyreadstat to read data
data_label_df['FADUR'] = (data_label_df['FADUR'].apply(iso8601_duration_to_timedelta)).astype(str).str.replace('0 days ', '')
data_label_df = clean_dataframe(data_label_df)                  
data_label_df

In [70]:
display(data_label_df.head())
print(f'{data_label_df.shape[0]} records')
print(f'{data_label_df["USUBJID"].nunique()} patients with avg of {data_label_df.shape[0]/data_label_df["USUBJID"].nunique():.0f} records/patient')

,USUBJID,FATEST,FACAT,FADTC,FASTRESN,FADUR,INSSTYPE,INSNMBOL,INSEXBOL,DXTRT,DEVICE
0,1335,BASAL INSULIN,BASAL,2010-10-10 00:00:00,0.049999999999999996,00:05:00,basal_chunk,NaN,NaN,MEDTRONIC 770G IN AUTO MODE,AID
1,1335,BASAL INSULIN,BASAL,2010-10-10 00:05:00,0.049999999999999996,00:05:00,basal_chunk,NaN,NaN,MEDTRONIC 770G IN AUTO MODE,AID
2,1335,BASAL INSULIN,BASAL,2010-10-10 00:10:00,0.049999999999999996,00:05:00,basal_chunk,NaN,NaN,MEDTRONIC 770G IN AUTO MODE,AID
3,1335,BOLUS INSULIN,BOLUS,2010-10-10 00:14:35,0.08,00:00:00,normal,0.08,NaN,MEDTRONIC 770G IN AUTO MODE,AID
4,1335,BASAL INSULIN,BASAL,2010-10-10 00:15:00,0.049999999999999996,00:05:00,basal_chunk,NaN,NaN,MEDTRONIC 770G IN AUTO MODE,AID


2866120 records
220 patients with avg of 13028 records/patient


In [31]:
display(data_label_df.value_counts(subset=['DEVICE', 'DXTRT','FACAT','FATEST'],sort=False))
display(data_label_df.value_counts(subset=['DXTRT','FATEST', 'INSSTYPE'],sort=False))

DEVICE  DXTRT                             FACAT  FATEST         
AID     MEDTRONIC 670G IN AUTO MODE       BASAL  BASAL FLOW RATE     150466
                                                 BASAL INSULIN       255796
                                          BOLUS  BOLUS INSULIN         6769
        MEDTRONIC 770G IN AUTO MODE       BASAL  BASAL FLOW RATE         73
                                                 BASAL INSULIN        24374
                                          BOLUS  BOLUS INSULIN        14960
        TANDEM T:SLIM X2 WITH CONTROL IQ  BASAL  BASAL FLOW RATE     850786
                                                 BASAL INSULIN      1490441
                                          BOLUS  BOLUS INSULIN        72455
Name: count, dtype: int64

DXTRT                             FATEST           INSSTYPE                  
MEDTRONIC 670G IN AUTO MODE       BASAL FLOW RATE                                150466
                                  BASAL INSULIN    automated                     137349
                                                   automated_chunk                65045
                                                   scheduled                        198
                                                   scheduled_chunk                46732
                                                   suspend                          695
                                                   suspend_chunk                   4809
                                                   temp                               5
                                                   temp_chunk                       963
                                  BOLUS INSULIN    dual/square_chunk                201
                                          

In [73]:
d = data_label_df.loc[(data_label_df.DXTRT == 'MEDTRONIC 670G IN AUTO MODE') & (data_label_df.FATEST == 'BOLUS INSULIN') &(data_label_df.INSSTYPE != 'normal')]
display(d.head(10))
d = data_label_df.loc[(data_label_df.DXTRT == 'TANDEM T:SLIM X2 WITH CONTROL IQ') & (data_label_df.FATEST == 'BOLUS INSULIN') & (data_label_df.INSSTYPE != 'normal')]
display(d.head(10))

,USUBJID,FATEST,FACAT,FADTC,FASTRESN,FADUR,INSSTYPE,INSNMBOL,INSEXBOL,DXTRT,DEVICE
161816,1497,BOLUS INSULIN,BOLUS,2010-07-26 19:10:42,,00:05:00,square_chunk,0.0,0.32,MEDTRONIC 670G IN AUTO MODE,AID
161818,1497,BOLUS INSULIN,BOLUS,2010-07-26 19:15:42,,00:05:00,square_chunk,NaN,0.32,MEDTRONIC 670G IN AUTO MODE,AID
161820,1497,BOLUS INSULIN,BOLUS,2010-07-26 19:20:42,,00:05:00,square_chunk,0.0,0.32,MEDTRONIC 670G IN AUTO MODE,AID
161822,1497,BOLUS INSULIN,BOLUS,2010-07-26 19:25:42,,00:05:00,square_chunk,NaN,0.32,MEDTRONIC 670G IN AUTO MODE,AID
161824,1497,BOLUS INSULIN,BOLUS,2010-07-26 19:30:42,,00:05:00,square_chunk,0.0,0.32,MEDTRONIC 670G IN AUTO MODE,AID
161826,1497,BOLUS INSULIN,BOLUS,2010-07-26 19:35:42,,00:05:00,square_chunk,NaN,0.32,MEDTRONIC 670G IN AUTO MODE,AID
161828,1497,BOLUS INSULIN,BOLUS,2010-07-26 19:40:42,,00:05:00,square_chunk,0.0,0.32,MEDTRONIC 670G IN AUTO MODE,AID
161830,1497,BOLUS INSULIN,BOLUS,2010-07-26 19:45:42,,00:05:00,square_chunk,NaN,0.32,MEDTRONIC 670G IN AUTO MODE,AID
161832,1497,BOLUS INSULIN,BOLUS,2010-07-26 19:50:42,,00:05:00,square_chunk,0.0,0.32,MEDTRONIC 670G IN AUTO MODE,AID
161834,1497,BOLUS INSULIN,BOLUS,2010-07-26 19:55:42,,00:05:00,square_chunk,NaN,0.32,MEDTRONIC 670G IN AUTO MODE,AID


,USUBJID,FATEST,FACAT,FADTC,FASTRESN,FADUR,INSSTYPE,INSNMBOL,INSEXBOL,DXTRT,DEVICE
454555,1012,BOLUS INSULIN,BOLUS,2009-11-22 14:44:24,,00:05:00,dual/square_chunk,0.0,0.21,TANDEM T:SLIM X2 WITH CONTROL IQ,AID
454558,1012,BOLUS INSULIN,BOLUS,2009-11-22 14:49:24,,00:05:00,dual/square_chunk,3.6,0.21,TANDEM T:SLIM X2 WITH CONTROL IQ,AID
454561,1012,BOLUS INSULIN,BOLUS,2009-11-22 14:54:24,,00:05:00,dual/square_chunk,0.0,0.21,TANDEM T:SLIM X2 WITH CONTROL IQ,AID
454564,1012,BOLUS INSULIN,BOLUS,2009-11-22 14:59:24,,00:05:00,dual/square_chunk,3.6,0.21,TANDEM T:SLIM X2 WITH CONTROL IQ,AID
454567,1012,BOLUS INSULIN,BOLUS,2009-11-22 15:04:24,,00:05:00,dual/square_chunk,0.0,0.21,TANDEM T:SLIM X2 WITH CONTROL IQ,AID
454570,1012,BOLUS INSULIN,BOLUS,2009-11-22 15:09:24,,00:05:00,dual/square_chunk,3.6,0.21,TANDEM T:SLIM X2 WITH CONTROL IQ,AID
454573,1012,BOLUS INSULIN,BOLUS,2009-11-22 15:14:24,,00:05:00,dual/square_chunk,0.0,0.21,TANDEM T:SLIM X2 WITH CONTROL IQ,AID
454576,1012,BOLUS INSULIN,BOLUS,2009-11-22 15:19:24,,00:05:00,dual/square_chunk,3.6,0.21,TANDEM T:SLIM X2 WITH CONTROL IQ,AID
454579,1012,BOLUS INSULIN,BOLUS,2009-11-22 15:24:24,,00:05:00,dual/square_chunk,0.0,0.21,TANDEM T:SLIM X2 WITH CONTROL IQ,AID
454582,1012,BOLUS INSULIN,BOLUS,2009-11-22 15:29:24,,00:05:00,dual/square_chunk,3.6,0.21,TANDEM T:SLIM X2 WITH CONTROL IQ,AID


In [53]:
#check if INSNMBOL for dual/square_chunk and  dual/square  (INSSTYPE) sum to the same value of INSEXBOL
d = data_label_df.loc[data_label_df.INSSTYPE.isin(['dual/square_chunk', 'dual/square']) & (data_label_df.DXTRT=='TANDEM T:SLIM X2 WITH CONTROL IQ')]
d.groupby(['INSSTYPE']).agg({'INSNMBOL':'sum', 'INSEXBOL':'sum'})

,INSNMBOL,INSEXBOL
INSSTYPE,,
dual/square,51.74,1.57
dual/square_chunk,36085.52,3284.03


 - Only 220 patients are extracted while the dataset contains 561 patients
 - 770G has no extended boluses, this is probably the reason why no chuking of boluses was performed
 
 - In some cases, there seem to be both cunked and unchunked data
    - while there are normal boluses (normal, standard, standard correction,), no chunked versions of these exist (probably only basal and extended are chunked)
    - 670G: dual/square_chunk and square_chunk exist (difference unclear) but no unchunked versions exist
- Boluses are split in extended and unextended columns
 - INSNMBOL alternates between 0 and NaN (670G)
 - In TANDEM: We see that INSNMBOL column repeats a standard bolus 
  - The sum of chunked boluses does not match the amount of unchunked boluses
 

As it seems, the chunking converts extended boluses and basal into 5 minute chunks and appends them to the original dataframe,
then it removes all rows that have a duration > 5.5 minutes.